# Baseline LLM

## Imports

In [ ]:
!pip install langchain==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install accelerate==0.21.0 transformers==4.31.0 tokenizers==0.13.3
# !pip install bitsandbytes==0.40.0 einops==0.6.1
# !pip install xformers==0.0.22.post7
# !pip install faiss-gpu==1.7.1.post3
# !pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux

In [ ]:
# %pip install langchain openai langchain_community langchain_openai huggingface_hub

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Get LLM

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
model_id = "IlyaGusev/saiga_llama3_8b"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
# )

In [ ]:
device

'cuda:0'

In [ ]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map=device,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

Prepare model, tokenizer: 811.294 sec.


In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
        max_new_tokens=500)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Prepare pipeline: 1.669 sec.


In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=20,
        eos_token_id=tokenizer.eos_token_id)
    time_2 = time()
    # print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    return sequences

In [ ]:
from tqdm import tqdm

In [ ]:
instruct = """
Ты очень талантливый новостной журналист, и твоя задача написать правдободобную новостную статью в журнал по заданным темам.

Напиши длинную статью-новость со следующими тематиками: {}

Твоя статья:
"""

In [ ]:
res = test_model(tokenizer,
           query_pipeline,
           instruct.format('Погода'))
res_ = [i['generated_text'].split('Твоя статья:')[1] for i in res]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


In [ ]:
label_map = {
    'Красота': 0,
    'Политика': 1,
    'Экономика': 2,
    'Мода': 3,
    'Общество': 4,
    'Закон и право': 5,
    'Кино': 6,
    'Телевидение': 7,
    'Персоны': 8,
    'События': 9,
    'Бренды': 10,
    'Наука': 11,
    'Гаджеты': 12,
    'Соцсети': 13,
    'Дом': 14,
    'Технологии': 15,
    'Транспорт': 16,
    'Погода': 17,
    'Рецепты': 18,
    'Опросы': 19,
    'Головоломки': 20
}

In [14]:
extra_combinations = {
    'Экономика и Политика': [],
    'Экономика и Гаджеты и Технологии': [],
    'Дом и Технологии': [],
    'Наука и Транспорт': [],
    'Персоны и Кино': [],
    'Гаджеты и Соцсети': [],
    'Транспорт и Авто и Городское движение': [],
}

In [15]:
# [4, 5]        12886
# [4, 8]        10442
# [1, 5]        10179
# [2]            7786
# [7]            7668
# [13]           7455
# [5]            7147
# [6, 7]         6968
# [11]           6346
# [14]           4144
# [12, 15]       2965
# [8, 9]         2931
# [0, 3, 10]     2890
# [17]            802
# [19]            763
# [18]            751
# [20]            749
# [16]            514
# [2, 7]          443
# [5, 8]          437
# [6]             173

In [16]:
# topics_extra = {'Погода': [],
#                 'Рецепты': [],
#                 'Опросы': [],
#                 'Головоломки': []}

for topic in tqdm(list(extra_combinations.keys())):
    for num in tqdm(range(0, 20)):
        res_gen_news = test_model(tokenizer,
           query_pipeline,
           instruct.format(topic))
        res_ = [i['generated_text'].split('Твоя статья:')[1] for i in res_gen_news]
        extra_combinations[topic].extend(res_)

 45%|████▌     | 9/20 [04:40<05:43, 31.19s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(

100%|██████████| 7/7 [1:12:10<00:00, 618.65s/it]


In [17]:
topic

'Транспорт и Авто и Городское движение'

In [18]:
import pandas as pd

extra_df = pd.DataFrame(extra_combinations)

In [19]:
extra_df

,Экономика и Политика,Экономика и Гаджеты и Технологии,Дом и Технологии,Наука и Транспорт,Персоны и Кино,Гаджеты и Соцсети,Транспорт и Авто и Городское движение
0,\nЗагрузка и задержка в доставке грузов: причи...,\n«Уникальные решения для цифры в эпоху 5G»\n\...,\n«Экологическая архитектура: как современные ...,"\nНазвание новости: ""Новый тип двигателя для к...","\nУважаемые читатели, сегодня мы будем говорит...","\nТеперь мы знаем, как использовать соцсети дл...",\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...
1,"\n```\n""Ударный подъем: как экономика Китая и ...",\nЭкономико-технологический сектор развивается...,\n«Новый этап в развитии домов будущего - техн...,"\nНазвание статьи: ""Ученые обнаружили новые ис...","\n""Супер-звезда"" Кейт Уилсона стала новой ""Сер...","\n""Мир гаджетов и соцсетей""\n""Социальные сети ...",\n```\nМиллиарды на улицах: как изменения в го...
2,"\n```\nУважительное мнение экспертов: ""Пандеми...","\n""Мировой рынок смартфонов вновь обогнался но...","\nНазвание новостной статьи: ""Дом для будущего...","\n""Великие открытия, которые меняют нашу жизнь...",\nМир кино: что готовить на праздничном столе?...,"\n""Вероятность того, что вы потеряете свои дан...","\n```\nНазвание новостной статьи: ""Новый год в..."
3,\nНазвание страницы: Экономика и Политика\n\nН...,\n«Минскская экономика и гаджеты: новости и пе...,"\nТема: ""Уникальные технологии, которые изменя...",\n«Транспорт будущего – это не только новые ма...,"\n""Лидеры Голливуда: от кастинга до премер""\n!...","\nНазвание статьи: ""Новейшие гаджеты и соцсети...",\nВладивосток: Новые тенденции в городском дви...
4,\nЗдесь можно будет узнать о последних новостя...,\nЭкосоответствующие тенденции в гаджет-индуст...,"\nДом и Технологии\n\nТема 1: ""Дом - это не то...",\nНаука и Транспорт\n\nУченые из университета ...,"\n""Великий актер Александр Сокуровский: жизнь ...",\n```\nТЕХНОЛОГИЧЕСКИЙ МАРКЕТ 2022: ТЕНДЕНЦИИ ...,"\n1. Название и заголовок: ""Новый мост в город..."
...,...,...,...,...,...,...,...
395,\nНазвание статьи: «Соц-экономические вызовы д...,"\nTitle: ""Smart Home Automation: The Future of...",\nУвеличивая энергоэффективность с помощью инн...,\nУскоряя поиски новых материалов и технологий...,\nНазвание статьи: Новые звезды на горизонте: ...,\n«Технотрон 5000: новый гаджет для любого»\n\...,\nТранспорт и Авто: Тенденции в городском движ...
396,\nУспехи в области экономики и политики: что м...,\n```\nУгроза для мировой экономики: почему це...,\nУникальные технологии в строительном процесс...,"\nЗабавный виртуальный мир, полный чудес, науч...",\nТЕМ: Персоны и Кино\n\nВладимир Кличко: отхо...,\n«Технологий и соцсетей: как гаджеты и соцсет...,\n```\nЗадание 3: Транспорт и Авто и Городское...
397,"\n```\nСегодня 10 марта 2016 года, 13:00\nМоск...",\nЭкономика и Гаджеты: новый вектор экономичес...,\nЗагрузка в городской системе общественного т...,"\nНазвание: ""Ученые открывают новые способы эл...","\n""Сенсационная карьера актера Джеймса Кэрролл...",\nЗагруженность новыми технологиями: как гадже...,"\n""Сегодня в наш город была запущена новая про..."
398,\nЗагрузка портико\n\nВашим авторам удалось пр...,"\n```\nТехнологиям, которые меняют мир\n```\n\...","\nДом, который сам себе управляет: как искусст...",\n«Новая революция в науке: как ученые использ...,"\n""Великие актеры мирового кино: кто и где род...","\n""Социальные сети в наше время""\n\nСоциальные...",\nВчера на улицах нашего города прошел массовы...


In [20]:
extra_df.to_csv('./drive/MyDrive/VK_hack/extra_topics_multiple.csv')

In [ ]:
extra_df.iloc[199]

Погода         \nТеперь, когда уже совсем близко к зиме, нужн...
Рецепты        \nПоздно или рано, но когда дело доходит до во...
Опросы         \n«Непредсказуемые результаты: что мы научилис...
Головоломки    \nНазвание: Удивительные головоломки: что можн...
Name: 199, dtype: object

In [ ]:
topics_extra['Опросы']

['\nУраганные штормы разрушили ряд городов и поселков.\n\nСегодня, когда погода на ураганной волне, жители многих городов, особенно тех, которые расположены на побережья, испытывают огромные трудности.\n\nБезжалостный шторм обрушился на города, которые не успели подготовиться к таким сильным погодным условиям.\n\nТеперь в результате ураганных штормов разрушены ряд городов и поселков. Жители многих населенных пунктов оказались без крова и без средств к существованию.\n\nСоотношение погоды на ураганной волне к уникальным погодным условиям в разных странах мира, а также к различиям в погодных условиях в разных регионах одного и того же страны, является предметом исследования и анализа специалистами в области метеоотделения, климатологии и гидрометеорологии. В результате такого анализа, ученые могут получить данные о том, как именно погода на ураганной волне влияет на уникальные погодные условия в различных странах и регионах. Благодаря полученным данным, ученые могут разработать модели пр